# PESummary Workshop

# Introduction

PESummary is a code that is designed to display the results from all Bayesian analysis codes (both GW and non-GW specific) in HTML format. We hope that with these pages, the output from all Bayesian Analysis codes will be better interpreted and distributed and will be used by all areas of the scientific community. 

This workshop contains the following tutorials:

   1. [Linear Regression using the `pesummary.core` module](#Linear-Regression)
   2. [Gravitational wave analysis using the `pesummary.gw` module](#Gravitational-Wave-Analysis)
   3. [PESummary from the command line](#PESummary-cli)

## Installation

`pesummary` can be installed via `conda`, `pip` or `docker`. For full instructions on how to install `pesummary` inside your own virtual environment [see the installation instructions here](https://lscsoft.docs.ligo.org/pesummary/installation.html). For the purpose of this tutorial, we will `pip` install `pesummary` inside our Jupyter kernel

In [1]:
import sys

In [3]:
!{sys.executable} -m pip install bilby
!{sys.executable} -m pip install pesummary==0.1.6

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Linear Regression

### Generating the data

In order to get the results file, we will use `bilby` to generate the samples for us. We will be using the [tutorial](https://git.ligo.org/lscsoft/bilby/blob/master/examples/other_examples/linear_regression.py) given in the `bilby` examples (copied below)

In [4]:
from __future__ import division
import bilby
import numpy as np
import matplotlib.pyplot as plt

# A few simple setup steps
label = 'linear_regression'
outdir = 'outdir'
bilby.utils.check_directory_exists_and_if_not_mkdir(outdir)


# First, we define our "signal model", in this case a simple linear function
def model(time, m, c):
    return time * m + c


# Now we define the injection parameters which we make simulated data with
injection_parameters = dict(m=0.5, c=0.2)

# For this example, we'll use standard Gaussian noise

# These lines of code generate the fake data. Note the ** just unpacks the
# contents of the injection_parameters when calling the model function.
sampling_frequency = 10
time_duration = 10
time = np.arange(0, time_duration, 1 / sampling_frequency)
N = len(time)
sigma = np.random.normal(1, 0.01, N)
data = model(time, **injection_parameters) + np.random.normal(0, sigma, N)

# Now lets instantiate a version of our GaussianLikelihood, giving it
# the time, data and signal model
likelihood = bilby.likelihood.GaussianLikelihood(time, data, model, sigma)

# From hereon, the syntax is exactly equivalent to other bilby examples
# We make a prior
priors = dict()
priors['m'] = bilby.core.prior.Uniform(0, 5, 'm')
priors['c'] = bilby.core.prior.Uniform(-2, 2, 'c')

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', nlive=500,
    sample='unif', injection_parameters=injection_parameters, outdir=outdir,
    label=label)

12:40 bilby INFO    : Running bilby version: 0.5.0:
12:40 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.
12:40 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.
12:40 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.
12:40 bilby INFO    : Running for label 'linear_regression', output will be saved to 'outdir'
12:40 bilby INFO    : Search parameters:
12:40 bilby INFO    :   m = Uniform(minimum=0, maximum=5, name='m', latex_label='m', unit=None, boundary=None)
12:40 bilby INFO    :   c = Uniform(minimum=-2, maximum=2, name='c', latex_label='c', unit=None, boundary=None)
12:40 bilby INFO    : Single likelihood evaluation took 1.296e-04 s
12:40 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'unif', 'verbose': True, 'periodic': None, 'check_point_delta_

 4513| logz=-133.872 +/-  0.107 | dlogz:  0.100 >  0.1000000

12:41 bilby INFO    : Writing checkpoint file outdir/linear_regression_resume.pickle
12:41 bilby INFO    : Reading resume file outdir/linear_regression_resume.pickle
12:41 bilby INFO    : Succesfuly read resume file outdir/linear_regression_resume.pickle


 4513| logz=-133.777 +/-  0.106 | dlogz:  0.000 >  0.100


12:41 bilby INFO    : Sampling time: 0:00:28.580121
12:41 bilby INFO    : Summary of results:
nsamples: 5013
log_noise_evidence:    nan
log_evidence: -133.777 +/-  0.106
log_bayes_factor:    nan +/-  0.106



This has now created a `json` results called `linear_regression_result.json` in the directory `./outdir`.

In [5]:
!ls outdir/*.json

outdir/linear_regression_result.json


### Running PESummary

To create a webpage displaying this information we need to:

   1. Pass PESummary the results file
   2. Determine a directory to store the webpages

In [6]:
import os

path_to_file = os.path.join("outdir", "linear_regression_result.json")
webdir = "webpage"

We can now use the function `generate_webpage` to generate a webpage showing all of the information

In [7]:
def generate_webpage(samples, web_directory, label=None):
    from pesummary.core import command_line, inputs, finish                     
    from pesummary.core.file import meta_file                                       
    from cli import summarypages, summaryplots
             
    # Generate an argparse.Namespace object
    parser = command_line.command_line()
    opts = parser.parse_args([])
                                                                                
    # Pass PESummary the results file and the directory to
    # store the webpages
    # Pass PESummary the results file and the directory to
    # store the webpages
    opts.samples = samples
    if not isinstance(samples, list):
        opts.samples = [samples]
    opts.webdir = web_directory if not os.path.isdir(web_directory) else None
    opts.existing = web_directory if os.path.isdir(web_directory) else None
    
    # Pass PESummary a label
    opts.labels = label
    if label and not isinstance(label, list):
        opts.labels = [label]
    
    # Do low level checks of the inputs                                             
    args = inputs.Input(opts)                                                       
                                                                                
    # Generate all plots for the given results file                                 
    summaryplots.PlotGeneration(args)                                               
                                                                                
    # Generate the webpages for the given results file                              
    summarypages.WebpageGeneration(args)                                            
                                                                                
    # Generate the metafile                                                         
    meta_file.MetaFile(args)                                                        
                                                                                
    # Clean up the current working directory                                        
    finish.FinishingTouches(args)

In [8]:
generate_webpage(path_to_file, webdir)

/Users/Charlie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2019-05-13  12:41:16 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, baseurl=None, config=None, dump=False, email=None, existing=None, inj_file=None, labels=None, samples=['outdir/linear_regression_result.json'], save_to_hdf5=False, user='albert.einstein', verbose=False, webdir='webpage')
2019-05-13  12:41:16 PESummary INFO    : Extracting the information from outdir/linear_regression_result.json
2019-05-13  12:41:16 PESummary INFO    : Copying the files to webpage
2019-05-13  12:41:16 PESummary INFO    : Starting to generate plots
/Users/Charlie/anaconda3/lib/python3.6/site-packages/pesummary/core/plots/plot.py:42: RuntimeWarning: invalid value

We can now open the newly created webpage by opening `home.html`

In [9]:
!open ./webpage/home.html

As you can see, the code assigns a default label for our linear regression run. However, this is not ideal, because if we have multiple runs with multiple webpages, then from this label, we have no idea how each run is different. Therefore we might want to pass the label `first_attempt_at_creating_a_webpage`.

In [10]:
import shutil
shutil.rmtree(webdir)

In [11]:
label = "first_attempt_at_creating_a_webpage"

In [12]:
generate_webpage(path_to_file, webdir, label=label)

2019-05-13  12:41:46 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, baseurl=None, config=None, dump=False, email=None, existing=None, inj_file=None, labels=['first_attempt_at_creating_a_webpage'], samples=['outdir/linear_regression_result.json'], save_to_hdf5=False, user='albert.einstein', verbose=False, webdir='webpage')
2019-05-13  12:41:46 PESummary INFO    : Extracting the information from outdir/linear_regression_result.json
2019-05-13  12:41:46 PESummary INFO    : Copying the files to webpage
2019-05-13  12:41:46 PESummary INFO    : Starting to generate plots
/Users/Charlie/anaconda3/lib/python3.6/site-packages/pesummary/core/plots/plot.py:42: RuntimeWarning: invalid value encountered in true_divide
  correlated = np.correlate(y, y, mode="full") / norm
2019-05-13  12:41:50 PESummary INFO    : Finished generating the plots
2019-05-13  12:41:50 PESummary INFO    : Starting to generate webpages
2019-05-13  12:41:50 PESummary INFO    : Finished generatin

In [13]:
!open ./webpage/home.html

Now you can see that everything is labelled by our chosen label, which makes it really easy to now which webpage we are looking at.

### PESummary metafile

`pesummary` stores all information regarding the run in a specially created metafile. This metafile can be either directly downloaded from the webpages using the `results_file` download button, or can be viewed directly in the directory `webdir/samples/posterior_samples.json`.

In [14]:
import json

with open(os.path.join(webdir, "samples", "posterior_samples.json"), "r") as f:
    data = json.load(f)

The structure of the `pesummary` metafile can be viewed below:

In [15]:
print(data.keys())
print(data["posterior_samples"].keys())
print(data["posterior_samples"]["first_attempt_at_creating_a_webpage"].keys())

dict_keys(['posterior_samples'])
dict_keys(['first_attempt_at_creating_a_webpage'])
dict_keys(['parameter_names', 'samples'])


However, `pesummary` offers a nice little module to load in an existing metafile called `pesummary.core.existing`. This module contains the class `ExistingFile` which either requires the path to the `posterior_samples.json` file, or the web directory that you used to store the information regarding the run

In [16]:
from pesummary.core.file.existing import ExistingFile

# Passing ExistingFile the path to the results file
results_file_path = os.path.join("webpage", "samples", "posterior_samples.json")
f = ExistingFile(results_file_path)
print("Loaded in with the path to the results file")
print(f.existing_labels)
print(f.existing_parameters)

# Passing ExistingFile the path to the web directory
f = ExistingFile(webdir)
print("\nLoaded in with the the web directory")
print(f.existing_labels)
print(f.existing_parameters)

Loaded in with the path to the results file
['first_attempt_at_creating_a_webpage']
[['m', 'c', 'log_likelihood', 'log_prior']]

Loaded in with the the web directory
['first_attempt_at_creating_a_webpage']
[['m', 'c', 'log_likelihood', 'log_prior']]


### Lets change something

Lets see what happens when we change the prior on the gradient and y intercept:

In [17]:
from __future__ import division
import bilby
import numpy as np
import matplotlib.pyplot as plt

# A few simple setup steps
label = 'linear_regression'
outdir = 'outdir_different_prior'
bilby.utils.check_directory_exists_and_if_not_mkdir(outdir)


# First, we define our "signal model", in this case a simple linear function
def model(time, m, c):
    return time * m + c


# Now we define the injection parameters which we make simulated data with
injection_parameters = dict(m=0.5, c=0.2)

# For this example, we'll use standard Gaussian noise

# Now we define the injection parameters which we make simulated data with
injection_parameters = dict(m=0.5, c=0.2)

# For this example, we'll use standard Gaussian noise

# These lines of code generate the fake data. Note the ** just unpacks the
# contents of the injection_parameters when calling the model function.
sampling_frequency = 10
time_duration = 10
time = np.arange(0, time_duration, 1 / sampling_frequency)
N = len(time)
sigma = np.random.normal(1, 0.01, N)
data = model(time, **injection_parameters) + np.random.normal(0, sigma, N)

# Now lets instantiate a version of our GaussianLikelihood, giving it
# the time, data and signal model
likelihood = bilby.likelihood.GaussianLikelihood(time, data, model, sigma)

# From hereon, the syntax is exactly equivalent to other bilby examples
# We make a prior
priors = dict()
priors['m'] = bilby.core.prior.Uniform(0, 2, 'm')
priors['c'] = bilby.core.prior.Uniform(-2, 2, 'c')

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', nlive=500,
    sample='unif', injection_parameters=injection_parameters, outdir=outdir,
    label=label)

12:42 bilby INFO    : Running for label 'linear_regression', output will be saved to 'outdir_different_prior'
12:42 bilby INFO    : Search parameters:
12:42 bilby INFO    :   m = Uniform(minimum=0, maximum=2, name='m', latex_label='m', unit=None, boundary=None)
12:42 bilby INFO    :   c = Uniform(minimum=-2, maximum=2, name='c', latex_label='c', unit=None, boundary=None)
12:42 bilby INFO    : Single likelihood evaluation took 1.458e-04 s
12:42 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'unif', 'verbose': True, 'periodic': None, 'check_point_delta_t': 600, 'nlive': 500, 'first_update': None, 'walks': 20, 'npdim': None, 'rstate': None, 'queue_size': None, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': None, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 2.0, 'facc': 0.5, 'slices': 5, 'update_interval': 300, 'print_func': <bound method Dynesty._print_fu

 4134| logz=-137.033 +/-  0.099 | dlogz:  0.100 >  0.10000

12:42 bilby INFO    : Writing checkpoint file outdir_different_prior/linear_regression_resume.pickle
12:42 bilby INFO    : Reading resume file outdir_different_prior/linear_regression_resume.pickle
12:42 bilby INFO    : Succesfuly read resume file outdir_different_prior/linear_regression_resume.pickle


 4134| logz=-136.938 +/-  0.099 | dlogz:  0.000 >  0.100


12:42 bilby INFO    : Sampling time: 0:00:25.032271
12:42 bilby INFO    : Summary of results:
nsamples: 4634
log_noise_evidence:    nan
log_evidence: -136.938 +/-  0.099
log_bayes_factor:    nan +/-  0.099



In [18]:
!ls outdir_different_prior/*.json

outdir_different_prior/linear_regression_result.json


`pesummary` now allows us to add this second results file to the existing webpage that we have already generated so we can start to compare results

In [19]:
label = "different_prior"
path_to_file = os.path.join("outdir_different_prior", "linear_regression_result.json")

In [20]:
generate_webpage(path_to_file, webdir, label=label)

2019-05-13  12:42:42 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, baseurl=None, config=None, dump=False, email=None, existing='webpage', inj_file=None, labels=['different_prior'], samples=['outdir_different_prior/linear_regression_result.json'], save_to_hdf5=False, user='albert.einstein', verbose=False, webdir=None)
2019-05-13  12:42:42 PESummary INFO    : Extracting the information from outdir_different_prior/linear_regression_result.json
2019-05-13  12:42:42 PESummary INFO    : Copying the files to webpage
2019-05-13  12:42:42 PESummary INFO    : Starting to generate plots
/Users/Charlie/anaconda3/lib/python3.6/site-packages/pesummary/core/plots/plot.py:42: RuntimeWarning: invalid value encountered in true_divide
  correlated = np.correlate(y, y, mode="full") / norm
2019-05-13  12:42:48 PESummary INFO    : Finished generating the plots
2019-05-13  12:42:48 PESummary INFO    : Starting to generate webpages
2019-05-13  12:42:48 PESummary INFO    : Failed

In [21]:
!open ./webpage/home.html

As you can see we have added the second results file to the existing webpage and this has not only generated the 1d_histograms for both results files, but it has also generated comparison plots and comparison statistics to allow us to see how different the two runs actually are. Let us see how this has affected the metafile:

In [22]:
results_file_path = os.path.join("webpage", "samples", "posterior_samples.json")
f = ExistingFile(results_file_path)
print(f.existing_labels)
print(f.existing_parameters)

['different_prior', 'first_attempt_at_creating_a_webpage']
[['m', 'c', 'log_likelihood', 'log_prior'], ['m', 'c', 'log_likelihood', 'log_prior']]


We can see that the new results file has also been added to the metafile under that label different_prior.

### No existing webpage, no problem!

If you have not already generated a webpage to add your second results file too, then this is not a problem, because you can pass both results files and both labels directly to `pesummary` and it will take care of everything

In [23]:
shutil.rmtree(webdir)

In [24]:
path_to_file = [os.path.join("outdir_different_prior", "linear_regression_result.json"),
                os.path.join("outdir", "linear_regression_result.json")]
label = ["different_prior", "first_attempt_at_creating_a_webpage"]

In [25]:
generate_webpage(path_to_file, webdir, label=label)

2019-05-13  12:43:17 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, baseurl=None, config=None, dump=False, email=None, existing=None, inj_file=None, labels=['different_prior', 'first_attempt_at_creating_a_webpage'], samples=['outdir_different_prior/linear_regression_result.json', 'outdir/linear_regression_result.json'], save_to_hdf5=False, user='albert.einstein', verbose=False, webdir='webpage')
2019-05-13  12:43:17 PESummary INFO    : Extracting the information from outdir_different_prior/linear_regression_result.json
2019-05-13  12:43:17 PESummary INFO    : Extracting the information from outdir/linear_regression_result.json
2019-05-13  12:43:17 PESummary INFO    : Copying the files to webpage
2019-05-13  12:43:17 PESummary INFO    : Starting to generate plots
/Users/Charlie/anaconda3/lib/python3.6/site-packages/pesummary/core/plots/plot.py:42: RuntimeWarning: invalid value encountered in true_divide
  correlated = np.correlate(y, y, mode="full") / norm


In [26]:
!open ./webpage/home.html

In [27]:
results_file_path = os.path.join("webpage", "samples", "posterior_samples.json")
f = ExistingFile(results_file_path)
print(f.existing_labels)
print(f.existing_parameters)

['different_prior', 'first_attempt_at_creating_a_webpage']
[['m', 'c', 'log_likelihood', 'log_prior'], ['m', 'c', 'log_likelihood', 'log_prior']]


# Gravitational Wave Analysis

### Generating the data

You can choose your method of choice, but in order to save time, I have already run a Bayesian analysis on GW150914 using `bilby`. We are now able to create a webpage to display the results.

In [28]:
def generate_webpage(samples, web_directory, label=None, psd=None,
                     calibration=None, config=None, gracedb=None,
                     approximant="IMRPhenomPv2"):
    from pesummary.core import command_line, finish
    from pesummary.gw import inputs
    from pesummary.gw.command_line import insert_gwspecific_option_group
    from pesummary.gw.file import meta_file                                       
    from cli import summarypages, summaryplots
                                       
    # Generate an argparse.Namespace object
    parser = command_line.command_line()
    insert_gwspecific_option_group(parser)
    opts = parser.parse_args([])
                                                                                
    # Pass PESummary the results file and the directory to
    # store the webpages
    opts.samples = samples
    if not isinstance(samples, list):
        opts.samples = [samples]
    opts.webdir = web_directory if not os.path.isdir(web_directory) else None
    opts.existing = web_directory if os.path.isdir(web_directory) else None
    
    # Pass PESummary a label
    opts.labels = label
    if label and not isinstance(label, list):
        opts.labels = [label]
        
    # Pass PESummary the approximant that was used
    opts.approximant = approximant
    if approximant and not isinstance(approximant, list):
        opts.approximant = [approximant]
        
    # Pass PESummary the list of psds and calibration envelopes
    # that were used in the analysis
    if psd and isinstance(psd, dict):
        opts.psd = psd
    elif psd:
        raise Exception("Please provide the psd as a dictionary "
                        "with keys corresponding to the IFO")
    else:
        opts.psd = None

    if calibration and isinstance(calibration, dict):
        opts.calibration = calibration
    elif calibration:
        raise Exception("Please provide the calibration envelopes "
                        "as a dictionary with keys corresponding to "
                        "the IFO")
    else:
        opts.calibration = None

    # Pass PESummary the gracedb entry for the event
    opts.gracedb = gracedb
    # Specify that we are using the gw module
    opts.gw = True

    # Do low level checks of the inputs                                             
    args = inputs.GWInput(opts)
                                                                                
    # Generate all plots for the given results file                                 
    summaryplots.GWPlotGeneration(args)                                               
                                                                                
    # Generate the webpages for the given results file                              
    summarypages.GWWebpageGeneration(args)                                            
                                                                                
    # Generate the metafile                                                         
    meta_file.GWMetaFile(args)                                                        
                                                                                
    # Clean up the current working directory                                        
    finish.FinishingTouches(args)

Again, we need to decide where to put the data.

In [29]:
path_to_file = os.path.join("GW150914", "GW150914_result.json")
webdir = "webpage_gw"

Now, because we are looking at gravitational wave specific results files we will have had to pass `bilby` a PSD in order to calculate the likelihood. This PSD can be given straight to `pesummary` and plotted as part of the webpage.

In [30]:
psd_H = os.path.join("GW150914", "H1_PSD.dat")
psd_L = os.path.join("GW150914", "L1_PSD.dat")
psds = {"H1": psd_H, "L1": psd_L}

In [31]:
generate_webpage(path_to_file, webdir, label="IMRPhenomPv2",
                 gracedb="GW150914", psd=psds)

2019-05-13  12:43:52 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, approximant=['IMRPhenomPv2'], baseurl=None, calibration=None, config=None, dump=False, email=None, existing=None, gracedb='GW150914', gw=True, inj_file=None, labels=['IMRPhenomPv2'], psd={'H1': 'GW150914/H1_PSD.dat', 'L1': 'GW150914/L1_PSD.dat'}, samples=['GW150914/GW150914_result.json'], save_to_hdf5=False, sensitivity=False, user='albert.einstein', verbose=False, webdir='webpage_gw')
2019-05-13  12:43:52 PESummary INFO    : Extracting the information from GW150914/GW150914_result.json
2019-05-13  12:44:23 PESummary INFO    : Failed to grab calibration data from GW150914/GW150914_result.json
2019-05-13  12:44:23 PESummary INFO    : Copying the files to webpage_gw
2019-05-13  12:44:23 PESummary INFO    : Starting to generate plots
/Users/Charlie/anaconda3/lib/python3.6/site-packages/pesummary/core/plots/plot.py:42: RuntimeWarning: invalid value encountered in true_divide
  correlated = np.

In [32]:
!open webpage_gw/home.html

As you can see, the pages do look a bit different between the core and gw modules but there is the same general structure. Unlike the GW specific example, it is difficult to know what the 'important' plots are which can go on the results page hompage.

### PESummary MetaFile

The metafile is different also between the two modules because for the gw specific results file, we need to be able to store the approximant used, the psds and the calibration envelopes. This information can be extracted directly from the existing file is the same way as before, but by using the `pesummary.gw.file.existing.GWExistingFile` class

In [33]:
from pesummary.gw.file.existing import GWExistingFile

results_file_path = os.path.join("webpage_gw", "samples", "posterior_samples.json")
f = GWExistingFile(results_file_path)
print(f.existing_labels)
print(f.existing_parameters)
print(f.existing_approximant)
print(f.existing_psd)

['IMRPhenomPv2']
[['mass_1', 'mass_2', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'dec', 'ra', 'theta_jn', 'psi', 'luminosity_distance', 'phase', 'geocent_time', 'log_likelihood', 'mass_ratio', 'total_mass', 'chirp_mass', 'symmetric_mass_ratio', 'iota', 'spin_1x', 'spin_1y', 'spin_1z', 'spin_2x', 'spin_2y', 'spin_2z', 'chi_p', 'chi_eff', 'cos_tilt_1', 'cos_tilt_2', 'redshift', 'comoving_distance', 'mass_1_source', 'mass_2_source', 'total_mass_source', 'chirp_mass_source']]
['IMRPhenomPv2']
{'H1': [[0.0, 2.9344185916944965e-41], [0.25, 3.0132572842237593e-40], [0.5, 1.8651878168446349e-41], [0.75, 7.278876613888595e-42], [1.0, 4.59378412168825e-42], [1.25, 4.00841481784663e-42], [1.5, 4.49891335986368e-42], [1.75, 5.792955765075644e-42], [2.0, 7.766240955971107e-42], [2.25, 1.1079419317399076e-41], [2.5, 1.9802253426668537e-41], [2.75, 3.1686913276817864e-41], [3.0, 2.8159796903110693e-41], [3.25, 5.058541806021548e-41], [3.5, 1.9782336028867955e-40], [3.75, 8.437830957084603

### Lets change something (GW version)

Lets see what happens when we change the approximant used to analyse GW150914.

In [34]:
path_to_file = os.path.join("GW150914", "GW150914_different_approximant_result.json")

In [35]:
generate_webpage(path_to_file, webdir, label="IMRPhenomP",
                 gracedb="GW150914", psd=psds,
                 approximant="IMRPhenomP")

2019-05-13  12:46:01 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, approximant=['IMRPhenomP'], baseurl=None, calibration=None, config=None, dump=False, email=None, existing='webpage_gw', gracedb='GW150914', gw=True, inj_file=None, labels=['IMRPhenomP'], psd={'H1': 'GW150914/H1_PSD.dat', 'L1': 'GW150914/L1_PSD.dat'}, samples=['GW150914/GW150914_different_approximant_result.json'], save_to_hdf5=False, sensitivity=False, user='albert.einstein', verbose=False, webdir=None)
2019-05-13  12:46:01 PESummary INFO    : Extracting the information from GW150914/GW150914_different_approximant_result.json
2019-05-13  12:46:32 PESummary INFO    : Failed to grab calibration data from GW150914/GW150914_different_approximant_result.json
2019-05-13  12:46:33 PESummary INFO    : Copying the files to webpage_gw
2019-05-13  12:46:33 PESummary INFO    : Starting to generate plots
2019-05-13  12:47:06 PESummary INFO    : Failed to generate waveform plot because Invalid argument


In [36]:
!open webpage_gw/home.html

### PESummary Metafile

In [37]:
results_file_path = os.path.join("webpage_gw", "samples", "posterior_samples.json")
f = GWExistingFile(results_file_path)
print(f.existing_labels)
print(f.existing_parameters)
print(f.existing_approximant)
print(f.existing_psd)

['IMRPhenomP', 'IMRPhenomPv2']
[['mass_1', 'mass_2', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'dec', 'ra', 'theta_jn', 'psi', 'luminosity_distance', 'phase', 'geocent_time', 'log_likelihood', 'mass_ratio', 'total_mass', 'chirp_mass', 'symmetric_mass_ratio', 'iota', 'spin_1x', 'spin_1y', 'spin_1z', 'spin_2x', 'spin_2y', 'spin_2z', 'chi_p', 'chi_eff', 'cos_tilt_1', 'cos_tilt_2', 'redshift', 'comoving_distance', 'mass_1_source', 'mass_2_source', 'total_mass_source', 'chirp_mass_source'], ['mass_1', 'mass_2', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'dec', 'ra', 'theta_jn', 'psi', 'luminosity_distance', 'phase', 'geocent_time', 'log_likelihood', 'mass_ratio', 'total_mass', 'chirp_mass', 'symmetric_mass_ratio', 'iota', 'spin_1x', 'spin_1y', 'spin_1z', 'spin_2x', 'spin_2y', 'spin_2z', 'chi_p', 'chi_eff', 'cos_tilt_1', 'cos_tilt_2', 'redshift', 'comoving_distance', 'mass_1_source', 'mass_2_source', 'total_mass_source', 'chirp_mass_source']]
['IMRPhenomP', 'IMRPhenomP

### No existing webpage, no problem (GW version)!

In the same sense as before, we can combine multiple GW specific results files into the same summarypage even when we have not already generated one in the past.

In [38]:
shutil.rmtree(webdir)

In [39]:
path_to_file = [os.path.join("GW150914", "GW150914_result.json"),
                os.path.join("GW150914", "GW150914_different_approximant_result.json")]
label = ["IMRPhenomPv2", "IMRPhenomP"]

In [40]:
psd_H = os.path.join("GW150914", "H1_PSD.dat")
psd_L = os.path.join("GW150914", "L1_PSD.dat")
psds = {"H1": [psd_H, psd_H], "L1": [psd_L, psd_L]}

In [41]:
generate_webpage(path_to_file, webdir, label=label,
                 gracedb="GW150914", psd=psds,
                 approximant=["IMRPhenomPv2", "IMRPhenomP"])

2019-05-13  12:48:52 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, approximant=['IMRPhenomPv2', 'IMRPhenomP'], baseurl=None, calibration=None, config=None, dump=False, email=None, existing=None, gracedb='GW150914', gw=True, inj_file=None, labels=['IMRPhenomPv2', 'IMRPhenomP'], psd={'H1': ['GW150914/H1_PSD.dat', 'GW150914/H1_PSD.dat'], 'L1': ['GW150914/L1_PSD.dat', 'GW150914/L1_PSD.dat']}, samples=['GW150914/GW150914_result.json', 'GW150914/GW150914_different_approximant_result.json'], save_to_hdf5=False, sensitivity=False, user='albert.einstein', verbose=False, webdir='webpage_gw')
2019-05-13  12:48:52 PESummary INFO    : Extracting the information from GW150914/GW150914_result.json
2019-05-13  12:49:24 PESummary INFO    : Extracting the information from GW150914/GW150914_different_approximant_result.json
2019-05-13  12:49:55 PESummary INFO    : Failed to grab calibration data from GW150914/GW150914_result.json
2019-05-13  12:49:55 PESummary INFO    : Fai

In [42]:
!open webpage_gw/home.html

# Lets do some real science

Now that we have shown what `pesummary` can do and how useful it can be in viewing the contents of a results file, we can start to do some actual science and compare the results from `pycbc_inference` and `bilby`. I have already ran both codes on `GW170729` and now lets use `pesummary` to compare the contents:

In [43]:
webdir = "comparison"

In [44]:
path_to_file = [os.path.join("GW170729", "GW170729_result.json"),
                os.path.join("GW170729", "gw170729_posteriors_thinned.hdf")]
label = ["bilby", "pycbc_inference"]

In [45]:
psd_H = os.path.join("GW150914", "H1_PSD.dat")
psd_L = os.path.join("GW150914", "L1_PSD.dat")
psds = {"H1": [psd_H, psd_H], "L1": [psd_L, psd_L]}

In [46]:
sys.path.insert(0, "/users/charlie/documents/cardiff/GitLab/pesummary")
generate_webpage(path_to_file, webdir, label=label,
                 gracedb="GW170729", psd=psds,
                 approximant=["IMRPhenomPv2", "IMRPhenomPv2"])

2019-05-13  13:01:07 PESummary INFO    : Command line arguments: Namespace(add_to_existing=False, approximant=['IMRPhenomPv2', 'IMRPhenomPv2'], baseurl=None, calibration=None, config=None, dump=False, email=None, existing=None, gracedb='GW170729', gw=True, inj_file=None, labels=['bilby', 'pycbc_inference'], psd={'H1': ['GW150914/H1_PSD.dat', 'GW150914/H1_PSD.dat'], 'L1': ['GW150914/L1_PSD.dat', 'GW150914/L1_PSD.dat']}, samples=['GW170729/GW170729_result.json', 'GW170729/gw170729_posteriors_thinned.hdf'], save_to_hdf5=False, sensitivity=False, user='albert.einstein', verbose=False, webdir='comparison')
2019-05-13  13:01:07 PESummary INFO    : Extracting the information from GW170729/GW170729_result.json
2019-05-13  13:02:11 PESummary INFO    : Extracting the information from GW170729/gw170729_posteriors_thinned.hdf
2019-05-13  13:02:59 PESummary WARNING : Phase it not given, we will be assuming that a reference phase of 0 to calculate all the spin angles
2019-05-13  13:03:56 PESummary I

In [48]:
!open comparison/home.html

# PESummary cli

Despite being able to be used from within a python shell, PESummary was designed to be used from the command line. PESummary offers the `summarypages` executable which does everything that we have done above in a single command. The `summarypages` executable takes the following command line arguments as options

In [49]:
!summarypages --help

/Users/Charlie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
usage: summarypages [-h] [-w DIR] [-b DIR] [-s SAMPLES [SAMPLES ...]]
                    [-c CONFIG [CONFIG ...]] [--email EMAIL] [--dump]
                    [--add_to_existing] [-e EXISTING]
                    [-i INJ_FILE [INJ_FILE ...]]
                    [--labels LABELS [LABELS ...]] [-v] [--save_to_hdf5]
                    [-a APPROXIMANT [APPROXIMANT ...]] [--sensitivity]
                    [--gracedb GRACEDB] [--psd PSD [PSD ...]]
                    [--calibration CALIBRATION [CALIBRATION ...]] [--gw]

optional arguments:
  -h, --help            show this help message and exit
  -w DIR, --webdir DIR  make page and plots in DIR
  -b DIR, --baseurl DIR
        

An example of a bash command that you would run for the above results file would be:

```bash
$ summarypages --webdir ./webpage_gw \
               --samples ./GW150914/GW150914_result.json \
               --approximant IMRPhenomPv2 \
               --psd H1:GW150914/H1_PSD.dat L1:GW150914/L1_PSD.dat \
               --labels IMRPhenomPv2 \
               --gracedb GW150914 \
               --gw
```

for a single results file or:

```bash
$ summarypages --webdir ./webpage_gw \
               --samples ./GW150914/GW150914_result.json ./GW150914/GW150914_different_approximant_result.json \
               --approximant IMRPhenomPv2 IMRPhenomP \
               --psd H1:GW150914/H1_PSD.dat H1:GW150914/H1_PSD2.dat L1:GW150914/L1_PSD.dat L1:GW150914/L1_PSD2.dat \
               --labels IMRPhenomPv2 IMRPhenomP \
               --gracedb GW150914 \
               --gw
```

for multiple results files.

# Questions

If you have any questions please do not hesitate to email me on `charlie.hoy@ligo.org`